<a href="https://colab.research.google.com/github/Yoon2596/kita_2404/blob/main/m5_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/ML_%EC%B6%94%EC%B2%9C1_%EC%9E%A0%EC%9E%AC%EC%9A%94%EC%9D%B8_%ED%98%91%EC%97%85_%ED%95%84%ED%84%B0%EB%A7%81(0802).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 확률적 경사 하강법(Stochastic Gradient Descent, SGD)

SGD를 이용한 행렬 분해는 주어진 행렬을 두 개의 저차원 행렬로 분해하여 데이터의 잠재 구조를 학습하는 방법입니다. 이 방법은 특히 대규모 데이터셋에서 효율적으로 행렬 분해를 수행하는 데 유용합니다. 다음은 이 방법의 의미와 주요 개념을 설명합니다.

행렬 분해의 목적은 주어진 행렬 R을 두 개의 저차원 행렬 P와 Q로 분해하여 다음과 같은 관계를 만족하는 것입니다:

$R≈P X Q^T$

여기서:
- R은 원본 행렬 (예: 사용자-아이템 평점 행렬)
- P는 사용자 잠재 요인 행렬
- Q는 아이템 잠재 요인 행렬

확률적 경사 하강법(SGD)의 개념
- SGD는 최적화 알고리즘으로, 목표 함수를 최소화하기 위해 매 반복마다 데이터 샘플의 일부분을 사용하여 모델 파라미터를 업데이트합니다. 이는 대규모 데이터셋에서도 빠르게 수렴할 수 있는 장점이 있습니다.

[ SGD를 이용한 행렬 분해의 과정 ]

1. 초기화
- $P$와 $Q$ 행렬을 무작위 값으로 초기화합니다.
- 예를 들어, $P$는 $m \times k$ 행렬, $Q$는 $n \times k$ 행렬로 설정합니다. 여기서 $k$는 잠재 요인의 수입니다.<br>

2. 오차 계산
- 실제 값 $r_{ij}$와 예측 값 $\hat{r}_{ij}$ = $P_i \cdot Q_j^T$의 차이를 계산합니다.

  $e_{ij}$=$r_{ij}$ − $\hat{r}_{ij}$<br>

3. 파라미터 업데이트
- SGD를 사용하여 $P$와 $Q$의 파라미터를 업데이트합니다.

  $P_i \leftarrow P_i + \eta \left( e_{ij} Q_j - \lambda P_i \right)$

  $Q_j \leftarrow Q_j + \eta \left( e_{ij} P_i - \lambda Q_j \right)$

  여기서 $\eta$는 학습률, $\lambda$는 정규화 파라미터입니다.<br>

4. 반복
- 오차가 수렴할 때까지 또는 정해진 반복 횟수만큼 2번과 3번 과정을 반복합니다.

Surprise 라이브러리의 SVD 클래스는 실제로는 일반적으로 이해되는 SVD(특이값 분해, Singular Value Decomposition)와는 다릅니다. Surprise 라이브러리에서 사용하는 SVD는 사실상 Latent Factor Model(잠재 요인 모델)로서, 행렬 분해(Matrix Factorization)와 SGD(확률적 경사 하강법)를 사용하여 사용자-아이템 평점 행렬을 분해하는 방식

In [ ]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정.
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])

# num_user와 num_item은 행렬 R의 행과 열의 개수를 저장합니다. 이 값들은 각각 사용자와 아이템의 수를 의미
num_user, num_item = R.shape

K = 3   # K는 잠재 요인의 차원을 설정하는 변수로 P, Q 행렬의 차원을 정의합니다

# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다
# 이는 생성되는 랜덤 숫자들이 평균 0을 중심으로 표준편차가 1/3인 정규분포를 따르도록 합니다
np.random.seed(1)
P = np.random.normal(scale=1/3., size=(num_user, K))    # 평균 0, 표준편차 1/3인 정규분포 난수
Q = np.random.normal(scale=1/3., size=(num_item, K))

In [ ]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 Null이 아닌 값의 위치 인덱스를 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [ ]:
# R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
# R에서 null값을 제외한 데이터의 행렬 인덱스 추출, non_zeros 리스트에 저장
import math

non_zeros = [(i, j, R[i, j]) for i in range(num_user) for j in range(num_item) if not math.isnan(R[i, j])]
non_zeros

[(0, 0, 4.0),
 (0, 3, 2.0),
 (1, 1, 5.0),
 (1, 3, 3.0),
 (1, 4, 1.0),
 (2, 2, 3.0),
 (2, 3, 4.0),
 (2, 4, 4.0),
 (3, 0, 5.0),
 (3, 1, 2.0),
 (3, 2, 1.0),
 (3, 3, 2.0)]

In [ ]:
# SGD를 이용하여 P와 Q를 반복적으로 업데이트하여 행렬 분해를 수행

steps = 1000    # SGD를 반복해서 업데이트 할 횟수
learning_rate = 0.01    # SGD 학습률
r_lambda = 0.01    # L2 Regularization 계수

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
        Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0:    # 50회 반복 할 때마다 오류 값 출력
        print("### iteration step : ", step, " rmse : ", rmse)
#

### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.4876723101369648
### iteration step :  100  rmse :  0.1564340384819247
### iteration step :  150  rmse :  0.07455141311978046
### iteration step :  200  rmse :  0.04325226798579314
### iteration step :  250  rmse :  0.029248328780878973
### iteration step :  300  rmse :  0.022621116143829466
### iteration step :  350  rmse :  0.019493636196525135
### iteration step :  400  rmse :  0.018022719092132704
### iteration step :  450  rmse :  0.01731968595344266
### iteration step :  500  rmse :  0.016973657887570753
### iteration step :  550  rmse :  0.016796804595895633
### iteration step :  600  rmse :  0.01670132290188466
### iteration step :  650  rmse :  0.01664473691247669
### iteration step :  700  rmse :  0.016605910068210026
### iteration step :  750  rmse :  0.016574200475705
### iteration step :  800  rmse :  0.01654431582921597
### iteration step :  850  rmse :  0.01651375177473524
### iterati

In [ ]:
# 원본 행렬과 비교 시 null이 아닌 값은 큰 차이가 나지 않으며 nulll인 값은 새로운 예측값으로 채워짐
pred_matrix = np.dot(P, Q.T)
print('원본 행렬:\n', R)
print()
print('예측 행렬:\n', np.round(pred_matrix, 3))

원본 행렬:
 [[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
